In [ ]:
# в этом ноутбуке строю зависимость прибыльности сделки от размера скользящего стоп-лосса при рандомных трейдах.
# код написал клод с моими минорными правками. 
# моя гипотеза была в том, что для адекватных размеров стоп-лосса (меньше 1%) чем меньше стоп-лосс тем лучше. 
# я этот вывод сделал на основе наблюдений, нескольких тестов и разглядывая как меняются распределения прибылей 
# (see stop-loss.ipynb). гипотеза клода была, что зависимость будет в виде перевернутой параболы. 
# результат получился, что моя гипотеза более верна, чем гипотеза клода, но на самом деле размер средней прибыли при
# рандомных трейдах меняется несущественно от размера стоп-лосса. 
# отличие этого эксперимента от того, что сделано в stop-loss.ipynb в том, что здесь код написал клод и он больше 
# похож на рандомные трейды в том плане, что симулируется рандомность. в то время как в stop-loss.ipynb я сделал 
# предположение, что вероятность открытия трейда в любой момент одинаковая. что не совсем так при торговле со 
# скользящим стоплоссом. например, если позиция открылась и цена не меняется, то позиция не закрывается. а если цена 
# поменялась в неудачную сторону, то позиция закрывается и появляется шанс заново открыть позицию. т.е. вероятность 
# открытия позиции при движении цен выше чем без движения. я предполагаю, что это не существенно влияет на результат и 
# выводы, однако, решил провести эксперимент, раз уж клод написал код. 
# кстати, симуляция, которую делает клод в своем коде тоже не обязательно верно отражает распределение вероятности 
# открытия позиции при автоматизированной торговле с экспертным советником.

In [ ]:
import pandas as pd
import numpy as np
import forecast
import matplotlib.pyplot as plt

df = forecast.get_df('BTCUSD', '2024-01-02 00:00', '1T')

In [ ]:
def backtest_trailing_stop(df, stop_loss_percent):
    position = 0
    entry_price = 0
    stop_loss = 0
    pnl = []
    
    for i in range(1, len(df)):
        # if i % 100000 == 0:
        #     print(f'progress: {100*i/len(df)}')
        if position == 0:
            # Случайный вход в позицию
            if np.random.random() < 0.5:  # 50% шанс входа в каждый период
                position = 1 if np.random.random() < 0.5 else -1
                entry_price = df['close'].iloc[i]
                stop_loss = entry_price * (1 - position * stop_loss_percent)
        else:
            # Проверка на срабатывание стоп-лосса
            if (position == 1 and df['low'].iloc[i] <= stop_loss) or \
               (position == -1 and df['high'].iloc[i] >= stop_loss):
                pnl.append((stop_loss - entry_price) * position)
                position = 0
            else:
                # Обновление трейлинг стоп-лосса
                if position == 1:
                    stop_loss = max(stop_loss, df['close'].iloc[i] * (1 - stop_loss_percent))
                else:
                    stop_loss = min(stop_loss, df['close'].iloc[i] * (1 + stop_loss_percent))
    
    # Закрытие позиции в конце периода, если она открыта
    if position != 0:
        pnl.append((df['close'].iloc[-1] - entry_price) * position)
    
    result = np.mean(pnl) if pnl else 0
    print(f'stop_loss_percent: {stop_loss_percent}, result {result}, mean over {len(pnl)}')
    return result

# Тестирование различных размеров стоп-лосса
stop_loss_range = np.logspace(-4, -2, 20)  # от 0.01% до 10%
results = []

for stop_loss in stop_loss_range:
    avg_pnl = backtest_trailing_stop(df, stop_loss)
    results.append((stop_loss, avg_pnl))

# Построение графика
stop_losses, avg_pnls = zip(*results)
plt.figure(figsize=(10, 6))
plt.semilogx(stop_losses, avg_pnls, marker='o')
plt.xlabel('Размер скользящего стоп-лосса (%)')
plt.ylabel('Средняя доходность')
plt.title('Зависимость средней доходности от размера скользящего стоп-лосса')
plt.grid(True)
plt.show()

# Вывод оптимального размера стоп-лосса
optimal_stop_loss = stop_losses[np.argmax(avg_pnls)]
print(f"Оптимальный размер стоп-лосса: {optimal_stop_loss:.4%}")
print(f"Максимальная средняя доходность: {max(avg_pnls):.4f}")

In [ ]:
# результат тут такой, что с ростом размера стоплосса в диапозоне от 0.01% до 0.5% идет плавное снижение средней 
# прибыльности сделки. после 0.5% прибыльность начинает скакать. я интерпретирую это как недостаточное количество 
# данных для эксперимента. типа случайности дают слишком большой шум и не усредняются.